# MCDC solver
## Purpose
Given a certain decision give the necessary test cases to fulfill the MCDC rule.
The solver supports only the booleans AND and OR. The Negation boolean can be ignored for solving purposses and can taken in account later on when the result is translated into the conditions. The solution for this particular decision should then be negated.
Exclusive ORs, between and the use of a variable more than one time in the same decision is not supported.

## Instruction
Run all the cells in the notebook with exception of the last one. The last cell offers the possibility to define the decision.
The conditions of the decision are replaced with a different letter for each condition.
The format of the decision is: starting with a letter followed by (if necessary) a boolean (& for AND, | for OR) and another letter.

In [2]:
import re
import sys

## Condition
The condition contains it's name and the preceding and following boolean.

In [3]:
class Condition:
    def __init__(self):
        self.name = ''
        self.precop = ''
        self.trailop = ''

## Testcase
Testcase describes the value of the conditions as a sequence of bits. Where 1 stands for true and 0 stands for false.
The outcome of the decision given the values of the different conditions is also stored as res.

In [4]:
class Testcase:
    def __init__(self, seq, res):
        self.seq = seq
        self.res = res

## Decision
The following object contains the functionality:
- to extract the different conditions and their boolean connectors from the given string. 
- to derive the test cases
- and get rid of the double sequences
- to present the test cases

In [5]:
class Decision:
    def __init__(self):
        self.conditions = list()
        self.testcases = list()
        
    def extract(self, raw):
        elem = list(raw)
        size = len(raw)
        i = 0
        while i < size:
            temp = Condition()
            temp.name = elem[i]
            if i == 0:
                temp.precop = '-'
            else:
                temp.precop = elem[i-1]
            if (i+1) == size:
                temp.trailop = '-'
            else:
                temp.trailop = elem[i+1]
            self.conditions.append(temp)
            i += 2
    
    def derivation(self):
        testcase = list()
        length = len(self.conditions)
        self.testcases = [None] * 2 * length
        for i in range(0, length):
            for x in range(0, length - (length - i)):
                if self.conditions[x].trailop == "&":
                    testcase.append("1")
                else:
                    testcase.append("0")
            testcase.append("1")
            for x in range(i+1, length):
                if self.conditions[x].precop == "&":
                    testcase.append("1")
                else:
                    testcase.append("0")
            self.testcases[i] = Testcase(''.join(testcase), "True")
            testcase[i]=("0")
            self.testcases[i + length] = Testcase(''.join(testcase), "False")
            testcase.clear()
    
    def reduce(self):
        li = list()
        temp = list()
        for testcase in self.testcases:
            if testcase.seq not in li:
                li.append(testcase.seq)
                temp.append(testcase)
        self.testcases.clear()
        self.testcases = temp
        
    def print_testcases(self):
        i = 1
        for x in self.testcases:
            print('Testcase ' + str(i) + ': ' + x.seq + '   ' + x.res)
            i += 1

### Custom made exception class

In [6]:
class DecisionError(Exception):
    def __init__(self, value):
        self.value = value
    def __str__(self):
        return repr(self.value)

## Solver
The solver checks whether the decision has the right layout and the letters are only used once.

In [10]:
def solve_decision(sequence, fileName=None):
    try:
        if re.match(r"^[A-Za-z]{1}((\||\&)[A-Za-z])*$", sequence) == None: 
            raise DecisionError('Wrong layout')
        seq_split = re.split("\||\&", sequence)
        if len(seq_split) != len(set(seq_split)):
            raise DecisionError('Duplicate letters')
        
    except DecisionError as e:
        if e.value == 'Wrong layout':
            print('Decision has to start with a letter and can be followed by combinations of a "|" or "&" and letter. Got: ', sequence)
        else:
            print('A letter can only be used once within a decision')
        raise
    if fileName:
        print("File saved as: ")
    decision = Decision()
    decision.extract(sequence)
    decision.derivation()
    decision.reduce()
    decision.print_testcases()
    

## Please give your input here and run the code
To save the result in a text file, give the filename as second value to the function solve_decision. The results are saved using the file name and a timestamp.

In [12]:
# Give here the decision you want to analyse
# The decision should contain at least one variable represented by a letter.
# The decision can contain a letter only onces
# The first letter can be followed by a combination of '|' or '&' and a letter.
# Example: 'A&B|C'

decision = 'A'

solve_decision(decision)
#solve_decision(decision, "result")

Testcase 1: 1   True
Testcase 2: 0   False
